In [6]:
# First read in file
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn import svm

trendingSet = pd.read_csv('../data/new-datasets/trending_dataset.csv')
nontrendingSet = pd.read_csv('../data/new-datasets/nontrending_dataset.csv')
trendingDf = pd.DataFrame(trendingSet)
nontrendingDf = pd.DataFrame(nontrendingSet)

subsetTrending = trendingDf.head(6000)
subsetNontrending = nontrendingDf.head(len(subsetTrending))

#Concatenate data sets
fullSubset = pd.concat([subsetTrending, subsetNontrending])
fullSubset = fullSubset.drop_duplicates(subset='video_id', keep='first')
fullSubset['tags_length'] = np.array([int(i) if(i!='[0]') else 0 for i in fullSubset['tags_length'].iloc[:]])

#create labels
trendingLabels = np.array(fullSubset['trending'].iloc[:])
fullSubset = fullSubset.drop(columns=['tags','trending','duration','video_id','video_title'], axis=1)
matrixData = np.array(fullSubset)

#Standardize the data
matrixData = matrixData.astype('float32')
matrixData -= np.mean(matrixData, axis=0)
matrixData /= np.std(matrixData, axis=0)

#Fit the  model
clf = svm.SVC(kernel='linear', C=0.01)
clf.fit(matrixData, trendingLabels)

#Make predictions
trendingTestSet = trendingDf.iloc[-400:].drop(columns=['tags','trending','duration','video_id','video_title'], axis=1)
nontrendingTestSet = nontrendingDf.iloc[-400:].drop(columns=['tags','trending','duration','video_id','video_title'], axis=1)
totaldf = pd.concat([trendingTestSet, nontrendingTestSet])
smallTestSet = np.array(totaldf)

#test set labels
trendingTestLabels = trendingDf.iloc[-400:]['trending']
nontrendingTestLabels = nontrendingDf.iloc[-400:]['trending']
totalTestlabels = np.concatenate([trendingTestLabels,nontrendingTestLabels])

predictions = clf.score(smallTestSet, totalTestlabels)
print(predictions)

0.5
